In [ ]:
import pandas as pd

In [ ]:
data_start = 0
data_end = 5000
df = pd.read_csv(
        "/Users/krasimirtrifonov/Documents/GitHub/Transformer-Neural-Network/dataset/EURUSD_Daily_200005300000_202405300000.csv",
        delimiter="\t")

# Extract the closing prices
closing = df["<CLOSE>"].iloc[data_start:data_end]

### Augmented Dickey-Fuller (ADF) test using the statsmodels library:

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Assume 'data' is your dataframe and '<CLOSE>' is the column of interest

# Perform Augmented Dickey-Fuller test on the <CLOSE> column
adf_result = adfuller(df['<CLOSE>'])

# Extract the test statistics and p-value
adf_statistic = adf_result[0]
p_value = adf_result[1]
critical_values = adf_result[4]

# Print the results
print(f'ADF Statistic: {adf_statistic:.4f}')
print(f'p-value: {p_value:.4f}')
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}: {value:.4f}')

        •	The p-value of 0.253 is higher than the usual significance levels (1%, 5%, or even 10%), meaning we fail to reject the null hypothesis of the test. This suggests that the data is non-stationary.
        •	The ADF statistic is greater than all critical values, further confirming that the series likely has a unit root (non-stationary).
    Let try to make it stationary by Differencing and we perform the ADF test   
        

In [ ]:
# Differencing
close_diff = df['<CLOSE>'].diff().dropna()

### *Apply ADF on Difference Close

In [ ]:
# Perform Augmented Dickey-Fuller test on the <CLOSE> column
adf_result = adfuller(close_diff)

# Extract the test statistics and p-value
adf_statistic = adf_result[0]
p_value = adf_result[1]
critical_values = adf_result[4]

# Print the results
print(f'ADF Statistic on Difference Close: {adf_statistic:.4f}')
print(f'p-value: {p_value:.4f}')
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}: {value:.4f}')

	•	The ADF statistic is much lower than the critical values, and the p-value is 0.0, meaning we can now reject the null hypothesis.
	•	This indicates that the differenced data is stationary.

###    -Now let check for Engle’s ARCH test to check for <strong>heteroscedasticity</strong>

In [ ]:
from statsmodels.stats.diagnostic import het_arch

# Perform Engle's ARCH test
arch_test_result = het_arch(close_diff)

# Extract the test statistic and p-value
arch_statistic = arch_test_result[0]
arch_p_value = arch_test_result[1]

# Print the results
print(f'ARCH Test Statistic on Close Diff: {arch_statistic:.4f}')
print(f'p-value: {arch_p_value:.4f}')

##	•	A low p-value (typically < 0.05) suggests heteroscedasticity (i.e., variance that changes over time).
### Let try on repeating diff

In [ ]:
close_diff_diff = close_diff.diff().dropna()
# Perform Engle's ARCH test on Repeating Close Diff
arch_test_result = het_arch(close_diff_diff)

# Extract the test statistic and p-value
arch_statistic = arch_test_result[0]
arch_p_value = arch_test_result[1]

# Print the results
print(f'ARCH Test Statistic on Repeating Close Diff: {arch_statistic:.4f}')
print(f'p-value: {arch_p_value:.9f}')

import matplotlib.pyplot as plt

# Plot the first 50 points of the second differenced data
plt.figure(figsize=(10, 6))

# Plot the second difference of <CLOSE>
plt.plot(close_diff_diff.head(50), marker='o', linestyle='-', label='Second Difference of <CLOSE>')

# Add plot title and labels
plt.title('First 50 Points of Second Difference <CLOSE>')
plt.xlabel('Index')
plt.ylabel('Second Difference Close')
plt.grid(True)  # Adds a grid for better readability
plt.legend()  # Shows the legend

# Display the plot
plt.show()

###	•	The extremely low p-value suggests that even after applying a second differencing, the data still exhibits heteroscedasticity (variance is not constant over time).
## It is Time to apply GARCH 

In [ ]:
import pandas as pd
from arch import arch_model

# Load your data (ensure the percent change data is properly calculated)
# Assuming 'data' is the DataFrame with a 'CLOSE_PERCENT_CHANGE' column

# Prepare the percent change data for GARCH modeling
garch_data = close_diff

# Fit a GARCH(1, 1) model on the percent change data
garch_model = arch_model(garch_data, vol='Garch', p=1, q=1)
garch_fitted = garch_model.fit(disp="off")

# Summarize the model
print(garch_fitted.summary())

# Forecast future volatility (for the next 5 days, for example)
forecast = garch_fitted.forecast(horizon=5)
print(forecast.variance[-1:])  # This gives the predicted variance (volatility) for the next 5 periods

The results of your GARCH(1,1) model are as follows, with some key interpretations and next steps:

GARCH Model Summary:

Mean Model:

	•	mu: The mean (constant) of the series is approximately 0.000024, with a p-value of 0.729, which indicates that the mean is not statistically significant.

Volatility (GARCH) Model:

	•	omega: 1.0101 \times 10^{-6} (p-value = 0.000) – This is the baseline level of variance, and it is statistically significant.
	•	alpha[1]: 0.0500 (p-value = 0.000) – This measures the influence of past shocks (squared returns or volatility) on future volatility. The significance indicates that past volatility influences future volatility.
	•	beta[1]: 0.9300 (p-value = 0.000) – This measures the persistence of volatility. A value close to 1 indicates high persistence, meaning that volatility tends to remain elevated for longer periods.

Key Observations:

	1.	High persistence of volatility: With a beta of 0.93, volatility is persistent, meaning that once volatility rises, it tends to remain high for several periods.
	2.	Statistical significance: Both the alpha and beta coefficients are highly significant, meaning the GARCH model is capturing the volatility structure well.
	3.	Normal Distribution: The residuals (errors) are assumed to follow a normal distribution, but other distributions (like Student’s t) might improve the fit.

Forecast:

The model produces forecasts for the next 5 periods of volatility:

	•	The forecasted volatility (variance) starts at 0.000021 and increases slightly to 0.000023 by the 5th period.

Convergence Warning:

	•	The message "Inequality constraints incompatible" suggests that the model did not converge fully. This could mean:
	•	The starting values or constraints may need adjustment.
	•	Trying different distributions (e.g., Student’s t instead of normal) might help improve model fit.

Next Steps:

	1.	Refining the Model:
	•	Try changing the distribution assumption from normal to Student’s t to allow for heavier tails.
	•	Ensure proper convergence by adjusting model parameters or re-running the optimization with different settings.
	2.	Volatility Interpretation:
	•	The forecasted volatility is low (around 0.000021), indicating stable conditions in the short term. If you’re looking for predictive insights, this information could be useful in understanding future market behavior.
	3.	Residual Analysis:
	•	Check the residuals from the model to ensure no significant patterns or autocorrelation are left.

In [ ]:
# Rescale the data (multiply by 100)
scaled_data = close_diff * 100

# Fit the GARCH model with rescaled data
garch_model = arch_model(scaled_data, vol='Garch', p=1, q=1)
garch_rescaled_fitted = garch_model.fit(disp="off")

# Summarize the model
print(garch_rescaled_fitted.summary())


# Forecasting future volatility with rescaled data
forecast = garch_rescaled_fitted.forecast(horizon=5)
forecast_variance_rescaled = forecast.variance[-1:] / 100  # Scale back
print(forecast_variance_rescaled)

The recommendation to scale rather than normalize might seem unusual because in many machine learning contexts, normalization or standardization is a common preprocessing step. However, in the case of GARCH models (and other econometric models), there are reasons why scaling is preferred over normalization for better optimization and interpretation:

Why Scaling is Recommended in GARCH Models:

	1.	Econometric Models Work with Raw Value Ranges:
	•	In time series econometrics, especially with models like GARCH, the parameters are sensitive to the actual magnitude of the data. The variance and returns are calculated directly from the raw data, so the absolute scale of the input values affects how the optimizer estimates the volatility structure.
	•	Scaling by a constant factor (like multiplying by 100) maintains the relationships in the data, while simply normalizing (which changes the range to [0, 1]) may not preserve the underlying structure of volatility or returns in the same way.
	2.	Normalization Alters Statistical Properties:
	•	Normalizing the data (scaling it between 0 and 1 or z-scoring) would modify the mean and variance of the data. For models like GARCH, which rely on these properties to model volatility, such changes can disrupt the model’s ability to accurately estimate parameters.
	•	Scaling by a constant factor (e.g., multiplying by 100) preserves the mean-to-variance ratio and leaves the overall structure of the time series unchanged.
	3.	Convergence Stability:
	•	Optimization algorithms in econometric models often perform better when the data is in a reasonable range (not too small or too large). If the data has very small values (as seen with your original data), the optimizer might struggle to find a solution due to the precision required for floating-point calculations. Rescaling moves the data into a more manageable range, improving convergence.
	4.	Maintaining Interpretability:
	•	When you scale the data by a constant (e.g., 100), the output from the GARCH model can still be interpreted in terms of the original scale. You can easily scale the results (e.g., volatility forecasts) back to the original units.
	•	In contrast, normalization would make interpreting the model’s output more difficult, as it would no longer be in the original units of the data.

To Clarify:

	•	Scaling is a linear transformation that preserves the relationships between the data points (i.e., a proportional increase or decrease in values).
	•	Normalization changes the distribution of the data, which can affect models that rely on the original statistical properties of the series (like variance, mean, and residuals).

Example:

If your data values are small (e.g., 5.05 \times 10^{-5}), the optimizer might have trouble accurately estimating parameters because the changes between points are too subtle. Scaling by 100 simply stretches the data but keeps the relationship between data points intact.

Summary:

	•	Normalization could alter the statistical properties (mean, variance) that are critical for GARCH modeling.
	•	Scaling preserves the data’s structure while making the optimization easier and improving convergence.
	•	After fitting the model, you can easily scale the results back to the original units if needed.

Step 1: Plot Conditional Volatility Over Time

You can plot the conditional variance (volatility) over time, which gives insight into how well the model captures changing volatility.

In [ ]:
# Plot the conditional volatility over time (variance over time)
plt.figure(figsize=(10, 6))
# Plot the conditional volatility (square root of variance)
conditional_volatility = garch_rescaled_fitted.conditional_volatility
plt.plot(garch_rescaled_fitted.conditional_volatility, label='Conditional Volatility')
plt.title('Conditional Volatility Over Time (GARCH Model)')
plt.xlabel('Time')
plt.ylabel('Conditional Volatility')
plt.grid(True)
plt.legend()
plt.show()

Step 2: Residual Analysis

After fitting the GARCH model, we can check whether the residuals are behaving like white noise (uncorrelated and normally distributed). This is a key test to verify if the model has captured the volatility dynamics adequately.

In [ ]:
# Plot the standardized residuals
plt.figure(figsize=(10, 6))
resid = garch_rescaled_fitted.resid
plt.plot(garch_rescaled_fitted.resid, label='Standardized Residuals')
plt.title('Standardized Residuals from GARCH Model')
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
import numpy as np
# Test for autocorrelation in the residuals using the Ljung-Box test
ljung_box_test = acorr_ljungbox(garch_rescaled_fitted.resid, lags=[10], return_df=True)

print(ljung_box_test)
if  np.array(ljung_box_test.lb_pvalue)[0]  > 0.05:
    print('there’s no significant autocorrelation in the residuals, suggesting that the GARCH model has adequately captured the volatility dynamics')
else:
     print("GARCH is not adequate!")   

If the p-values from the Ljung-Box test are greater than 0.05, it means there’s no significant autocorrelation in the residuals, suggesting that the GARCH model has adequately captured the volatility dynamics.

In [ ]:
# Use a Q-Q plot to check if residuals are normally distributed
import scipy.stats as stats
stats.probplot(garch_rescaled_fitted.resid, dist="norm", plot=plt)
plt.title('Q-Q Plot of GARCH Model Residuals')
plt.show()

The Q-Q plot compares the distribution of the residuals to a theoretical normal distribution. In an ideal case where the residuals are normally distributed, the points should fall along the diagonal line.

In this case, deviations from the line, especially in the tails, suggest that the residuals might not be perfectly normally distributed. This could indicate the presence of fat tails (kurtosis) or skewness, which are common in financial time series data.

In [ ]:
# Forecast future volatility for 5 periods
forecast_rescaled = garch_rescaled_fitted.forecast(horizon=5)

# Print forecasted variance
print("Forecasted variance over next 5 periods:")
print(forecast_rescaled.variance[-1:])

In [ ]:
from scipy.stats import kurtosis, skew

# Assuming 'resid_data' contains the residuals from your GARCH model
# Replace 'resid_data['Residuals']' with your own data if necessary

# Calculate Skewness
skewness = skew(resid)

# Calculate Kurtosis
kurt = kurtosis(resid)

# Print the results
print(f'Skewness: {skewness:.4f}')
print(f'Kurtosis: {kurt:.4f}')

	Skewness: 0.0247
	•	A skewness value close to zero suggests that the residuals are nearly symmetric. This means there’s no significant skewness in the data.
	Kurtosis: 2.47
	•	Kurtosis less than 3 indicates platykurtic behavior, meaning the residuals have lighter tails than a normal distribution. 
	    This suggests that extreme events (outliers) are less frequent compared to what a normal distribution would predict.
	•	The residuals appear to have little skewness, meaning they are relatively symmetric around the mean.
	•	The kurtosis is slightly lower than 3, indicating fewer extreme values (fat tails) compared to a normal distribution. 
	    This suggests that the normal distribution assumption might still hold, although it’s on the borderline.

Step 1: Fit GARCH Models with Different Distributions

You can fit the GARCH model with various distributions to determine the best fit:

In [ ]:
from arch import arch_model

# Assuming 'rescaled_data' contains your rescaled returns or percent changes
# Fit GARCH(1,1) with different distributions

# Normal distribution (default)
garch_normal = arch_model(scaled_data, vol='Garch', p=1, q=1, dist='normal').fit(disp="off")

# Student's t-distribution
garch_t = arch_model(scaled_data, vol='Garch', p=1, q=1, dist='t').fit(disp="off")

# Skewed Student's t-distribution
garch_skewt = arch_model(scaled_data, vol='Garch', p=1, q=1, dist='skewt').fit(disp="off")

# Print model summaries
print("Normal Distribution GARCH:")
print(garch_normal.summary())

print("\nStudent's t-Distribution GARCH:")
print(garch_t.summary())

print("\nSkewed t-Distribution GARCH:")
print(garch_skewt.summary())

Step 2: Compare the Distributions Using AIC

The AIC (Akaike Information Criterion) can be used to compare the different models. The model with the lowest AIC is considered the best fit.

In [ ]:
# Compare AIC values for the different models
print(f"Normal AIC: {garch_normal.aic}")
print(f"Student's t AIC: {garch_t.aic}")
print(f"Skewed t AIC: {garch_skewt.aic}")

	•	The Student’s t-distribution is likely capturing the fat tails better, which means that your data contains
	 more extreme values than a normal distribution would predict.
	 Since the Student’s t-distribution GARCH model has the best fit, let’s move forward with a deeper analysis. Here’s a breakdown of what we can do next:

1. Residual Analysis for the Student’s t-Distribution GARCH Model:

	•	We need to ensure that the residuals from the Student’s t-distribution GARCH model behave like white noise (i.e., no autocorrelation and are approximately normally distributed).

2. Forecast Volatility Using the Student’s t-Distribution GARCH Model:

	•	We can also forecast future volatility using this model to see how well it captures future uncertainty.

In [ ]:
# Plot the standardized residuals from the Student's t-distribution GARCH model
plt.figure(figsize=(10, 6))
plt.plot(garch_t.resid, label='Standardized Residuals (Student\'s t GARCH)')
plt.title('Standardized Residuals from Student\'s t GARCH Model')
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.grid(True)
plt.legend()
plt.show()

Step 2: Test for Autocorrelation in the Residuals

We’ll use the Ljung-Box test to check if there is any remaining autocorrelation in the residuals. Ideally, there should be no significant autocorrelation left after the GARCH model is applied.

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# Test for autocorrelation in the residuals from the Student's t GARCH model
ljung_box_test_t = acorr_ljungbox(garch_t.resid, lags=[10], return_df=True)

# Print the Ljung-Box test result
print(ljung_box_test_t)

In [ ]:
# Q-Q plot to check if the residuals are normally distributed
stats.probplot(garch_t.resid, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals from Student\'s t GARCH Model')
plt.show()

In [ ]:
# Forecast future volatility for the next 5 periods using the Student's t GARCH model
forecast_t = garch_t.forecast(horizon=5)

# Print forecasted variance
print("Forecasted variance over the next 5 periods:")
print(forecast_t.variance[-1:])

Normal
Forecasted variance over next 5 periods:
           h.1       h.2       h.3       h.4       h.5
6240  0.136139  0.137133  0.138124  0.139113  0.140099

t Student

Forecasted variance over the next 5 periods:
           h.1       h.2       h.3       h.4       h.5
6240  0.129124  0.129916  0.130706  0.131496  0.132284

3. Test for Stationarity (Augmented Dickey-Fuller Test)

We’ll apply the Augmented Dickey-Fuller (ADF) test to the residuals to check if they are stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform ADF test on GARCH residuals
adf_test = adfuller(garch_t.resid)

# Extract and print ADF test results
adf_statistic = adf_test[0]
adf_p_value = adf_test[1]
print(f'ADF Statistic: {adf_statistic}')
print(f'p-value: {adf_p_value}')
if adf_p_value < 0.05:
    print('p-value is less than 0.05, the residuals are stationary.')

In [ ]:
from statsmodels.stats.diagnostic import het_arch

# Perform Engle's ARCH test for heteroscedasticity on the residuals
arch_test_resid = het_arch(garch_t.resid)

# Extract the test statistic and p-value
arch_stat_resid = arch_test_resid[0]
arch_p_value_resid = arch_test_resid[1]

print(f'ARCH Test Statistic: {arch_stat_resid}')
print(f'p-value: {arch_p_value_resid}')
if arch_p_value_resid > 0.05:
    print("A p-value > 0.05 means that heteroscedasticity is no longer present in the residuals, indicating that GARCH has successfully reduced it.")
else:
    print("GARCH fail to reduce heteroscedasticity")
    

A p-value > 0.05 means that heteroscedasticity is no longer present in the residuals, indicating that GARCH has successfully reduced it.

Let’s move forward with more advanced GARCH models, such as GJR-GARCH and EGARCH, to better capture the complexity of your data and reduce heteroscedasticity.

Next Steps:

	1.	GJR-GARCH Model: This model accounts for asymmetric volatility, meaning it can capture the fact that negative shocks might have a larger impact on volatility than positive ones.
	2.	EGARCH Model: The Exponential GARCH model can handle leverage effects and asymmetry more effectively. It models the logarithm of the variance, allowing it to capture both positive and negative shocks in a flexible way.

Step-by-Step Implementation:

1. GJR-GARCH Model

In [ ]:
from arch import arch_model

# Fit a GJR-GARCH(1,1) model with Student's t-distribution
gjr_garch_model = arch_model(scaled_data, vol='Garch', p=1, q=1, o=1, dist='t').fit(disp="off")

# Print the GJR-GARCH model summary
print(gjr_garch_model.summary())

# Extract residuals from the GJR-GARCH model
gjr_garch_residuals = gjr_garch_model.resid

# Perform Engle's ARCH test on the GJR-GARCH residuals to check for heteroscedasticity
arch_test_gjr_resid = het_arch(gjr_garch_residuals)
print(f"GJR-GARCH Residuals - ARCH Test Statistic: {arch_test_gjr_resid[0]}, p-value: {arch_test_gjr_resid[1]}")

2. EGARCH Model

In [ ]:
# Fit an EGARCH(1,1) model with Student's t-distribution
egarch_model = arch_model(scaled_data, vol='EGARCH', p=1, q=1, dist='t').fit(disp="off")

# Print the EGARCH model summary
print(egarch_model.summary())

# Extract residuals from the EGARCH model
egarch_residuals = egarch_model.resid

# Perform Engle's ARCH test on the EGARCH residuals to check for heteroscedasticity
arch_test_egarch_resid = het_arch(egarch_residuals)
print(f"EGARCH Residuals - ARCH Test Statistic: {arch_test_egarch_resid[0]}, p-value: {arch_test_egarch_resid[1]}")

Using a Variational Autoencoder (VAE) to minimize variance can be a powerful approach to model time series data, especially for reducing volatility or variance in a latent space representation of the data. VAEs are generative models that aim to learn a probabilistic latent space from which data can be generated, and they can be used to create smoother or more regularized versions of time series data.

Key Idea Behind Using a VAE:

A VAE models the distribution of data points in a lower-dimensional latent space while enforcing a regularization constraint (usually using a Kullback-Leibler (KL) divergence term) to ensure the latent representations follow a known distribution (often a standard normal distribution). This can help in:
	•	Reducing volatility or variance in the time series data by mapping it to a smoother latent space.
	•	Generating new samples from the learned latent space, potentially with reduced variance.

Steps for Using a VAE on Time Series Data to Minimize Variance:

	1.	Data Preparation: The same as with neural networks, we’ll prepare the time series data into sequences (lags).
	2.	Define the VAE Model: The VAE will have an encoder that maps the input time series data into a latent space, and a decoder that reconstructs the time series from the latent space.
	3.	Train the VAE: The VAE will minimize both the reconstruction loss (mean squared error between the original and reconstructed data) and the KL divergence (to ensure the latent variables follow a normal distribution).
	4.	Generate or Transform Data: Once trained, the VAE can generate new time series with minimized variance by regularizing the latent space.

Step-by-Step Implementation:

Step 1: Data Preparation

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define a function to create sequences (lags) for time series data
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequence = data[i:i+seq_length]
        sequences.append(sequence)
    return torch.tensor(np.array(sequences), dtype=torch.float32)

# Example data (use your residuals or transformed data)
data = garch_t.resid.values

# Normalize the data (you can choose to standardize or normalize)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.reshape(-1, 1)).reshape(-1)

# Create sequences of data (lags)
seq_length = 5  # Number of lagged observations to use
X = create_sequences(data_scaled, seq_length)

Step 2: Define the VAE Model

We will define a simple Variational Autoencoder with an encoder and decoder. The encoder compresses the input data into a latent space, and the decoder reconstructs the data from the latent representation.

In [111]:
class HomoscedasticTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(HomoscedasticTransformer, self).__init__()
        
        # Encoder
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc_mu = nn.Linear(hidden_size, latent_size)  # Mean of latent space
        self.fc_logvar = nn.Linear(hidden_size, latent_size)  # Log variance of latent space

        # Decoder
        self.fc3 = nn.Linear(latent_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, input_size)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std  # Sample from latent space

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Initialize the VAE
input_size = seq_length
hidden_size = 50
latent_size = 10  # Latent space dimension
vae_model = HomoscedasticTransformer(input_size, hidden_size, latent_size)

Step 3: Loss Function and Training the VAE

The VAE loss function consists of two parts:

	1.	Reconstruction Loss: Ensures the output matches the input (minimizing reconstruction error).
	2.	KL Divergence: Ensures the latent space follows a standard normal distribution, regularizing the latent representation.

In [112]:
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = nn.MSELoss()(recon_x, x)  # Reconstruction loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  # KL divergence
    # Volatility reduction term: penalize high variance in the reconstructed data
    # variance_penalty = torch.var(recon_x)
    return recon_loss + kl_loss #+ 0.1*variance_penalty

# Training settings
optimizer = torch.optim.Adam(vae_model.parameters(), lr=0.001)
epochs = 100

# Training loop
for epoch in range(epochs):
    vae_model.train()
    
    # Forward pass
    recon_x, mu, logvar = vae_model(X)
    loss = vae_loss(recon_x, X, mu, logvar)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Variance loss:{torch.var(recon_x):.6f}')

Step 4: Use the Trained VAE to Generate Smoothed Data

Once trained, the VAE can generate new time series data by sampling from the latent space.

In [113]:
# Use the trained VAE to generate new (smoothed) data
vae_model.eval()
with torch.no_grad():
    reconstructed_data, _, _ = vae_model(X)
    reconstructed_data = reconstructed_data.numpy()

# Reverse the normalization
reconstructed_data_rescaled = scaler.inverse_transform(reconstructed_data)
# reconstructed_data_rescaled = scaler.fit_transform(reconstructed_data)

# Plot the original and reconstructed data
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
zzz = data[-len(reconstructed_data):]
reconstructed_data_rescaled_zero = reconstructed_data_rescaled[:,0]
# plt.plot(data[-len(reconstructed_data):], label='Original')
# plt.plot(reconstructed_data_rescaled.flatten(), label='Reconstructed (VAE)')
plt.plot(reconstructed_data_rescaled[:,0], label='Reconstructed (VAE)')
plt.legend()
plt.title('Original vs Reconstructed Time Series (VAE)')
plt.show()

In [85]:
np.savetxt("reconstructed_data_rescaled.txt", reconstructed_data_rescaled)

In [114]:
from termcolor import colored
# Perform Engle's ARCH test for heteroscedasticity on the residuals
arch_test_resid = het_arch(reconstructed_data_rescaled[:,0])

# Extract the test statistic and p-value
arch_stat_resid = arch_test_resid[0]
arch_p_value_resid = arch_test_resid[1]

print(f'ARCH Test Statistic: {arch_stat_resid}')
print(f'p-value: {arch_p_value_resid:.4f}')
if arch_p_value_resid > 0.05:
    print(colored("A p-value > 0.05 means that heteroscedasticity is no longer present in the residuals, indicating that VAE has successfully reduced it.",'red'))
else:
    print(f"VAE fail to reduce heteroscedasticity")